In [ ]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.init
import torch.optim as optim 
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import precision_score, confusion_matrix
import os
import shutil

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

: 

In [44]:
learning_rate = 0.001
epochs = 15
batch_size = 100

In [45]:
mnist_train = datasets.MNIST(root="./datasets/", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = datasets.MNIST(root="./datasets/", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

In [46]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [36]:
model = CNN().to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 학습
def train(model, train_loader, criterion, optimizer):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
            images, labels = images.to(device), labels.to(device) 

            optimizer.zero_grad()

            outputs = model(images)  
            loss = criterion(outputs, labels) 

            loss.backward()  
            optimizer.step()  

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')
        
# 모델 평가
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    all_targets = []
    all_predictions = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing", leave=False):
            images, labels = images.to(device), labels.to(device) 
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_targets.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total
    precision_per_class = precision_score(all_targets, all_predictions, average=None)
    precision_avg = precision_score(all_targets, all_predictions, average='macro')
    
    print(f'Accuracy of the model on the test images: {accuracy:.2f}%')
    
    return accuracy, precision_avg

# 결과 저장
def save_result(model_name = "CNN", accuracy = None, precision = None):    
    results_df = pd.DataFrame({
        'Model Name': [model_name],
        'Accuracy': [accuracy],
        'Precision': [precision]
    })
    results_df.to_csv('./result.csv', index=False)
    print("Results saved to './result.csv'")

# 모델 학습 및 평가 실행
train(model, train_loader, criterion, optimizer)
accuracy, precision = test(model, test_loader)

save_result("CNN", accuracy, precision)

Epoch [1/15], Loss: 0.1983


Epoch [2/15], Loss: 0.0552


Epoch [3/15], Loss: 0.0386


Epoch [4/15], Loss: 0.0287


Epoch [5/15], Loss: 0.0229


...

Epoch [12/15], Loss: 0.0080


Epoch [13/15], Loss: 0.0063


Epoch [14/15], Loss: 0.0052


Epoch [15/15], Loss: 0.0035


Accuracy of the model on the test images: 98.76%
Results saved to './result.csv'


In [47]:
def makedir(path): 
   try: 
        os.makedirs(path) 
   except OSError: 
       if not os.path.isdir(path): 
           raise

train_output_dir = './datasets/MNIST/jpeg90/train/'
test_output_dir = './datasets/MNIST/jpeg90/test/'

makedir(train_output_dir)
makedir(test_output_dir)

# MNIST 데이터셋 로드
mnist_dataset_train = datasets.MNIST(root="./datasets/", train=True, download=True)
mnist_dataset_test = datasets.MNIST(root="./datasets/", train=False, download=True)

# MNIST 데이터셋의 각 이미지를 JPEG로 변환 및 저장
for idx, (image, label) in enumerate(mnist_dataset_train):
    # 이미지 파일 이름 생성
    file_name = f"image_{idx}_label_{label}.jpg"
    output_file_path = os.path.join(train_output_dir, file_name)
    
    # JPEG로 저장 (퀄리티 90)
    image.convert('RGB').save(output_file_path, 'JPEG', quality=90)


# MNIST 데이터셋의 각 이미지를 JPEG로 변환 및 저장
for idx, (image, label) in enumerate(mnist_dataset_test):
    # 이미지 파일 이름 생성
    file_name = f"image_{idx}_label_{label}.jpg"
    output_file_path = os.path.join(test_output_dir, file_name)
    
    # JPEG로 저장 (퀄리티 90)
    image.convert('RGB').save(output_file_path, 'JPEG', quality=90)

In [61]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1), 
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

jpeg_90_train_dir = './datasets/MNIST/jpeg90/train/'
jpeg_90_test_dir = './datasets/MNIST/jpeg90/test/'

jpeg_90_train_loader = None
jpeg_90_test_loader = None

for i in range(10):
    train_source_dir = os.path.join(jpeg_90_train_dir, str(i))
    jpeg_90_train = datasets.ImageFolder(train_source_dir, transform=transform)
    jpeg_90_train_loader += DataLoader(jpeg_90_train, batch_size=64, shuffle=True)
    
    test_source_dir = os.path.join(jpeg_90_test_dir, str(i))
    jepg_90_test += datasets.ImageFolder(test_source_dir, transform=transform)
    jpeg_90_test_loader += DataLoader(jpeg_90_test, batch_size=64, shuffle=False)
    
# jpeg_90_train = datasets.ImageFolder(jpeg_90_train_dir, transform=transform)
# jpeg_90_test = datasets.ImageFolder(jpeg_90_test_dir, transform=transform)

# jpeg_90_train_loader = DataLoader(jpeg_90_train, batch_size=64, shuffle=True)
# jpeg_90_test_loader = DataLoader(jpeg_90_test, batch_size=64, shuffle=False)

jpeg_90_model = CNN().to(device)

train(jpeg_90_model, jpeg_90_train_loader, criterion, optimizer)
accuracy, precision = test(jpeg_90_model, jpeg_90_test_loader)

save_result("JPEG 90 CNN", accuracy, precision)

FileNotFoundError: Couldn't find any class folder in ./datasets/MNIST/jpeg90/train/0.